[![LibKet](../images/LibKet.png)](https://gitlab.com/mmoelle1/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/mmoelle1/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://mmoelle1.gitlab.io/LibKet/

---

# Tutorial \#1: Hands-on introduction to quantum computing with LibKet

## Getting started

**LibKet** is implemented as header-only C++14 library. It suffices to include the main header file `LibKet.hpp` in your application. This can take some time, stay tuned.

In [ ]:
#include "LibKet.hpp"

**LibKet**'s main functionality resides in the global namespace `LibKet` and in the nested namespaces `circuits`, `filters`, and `gates`.

Let's import their functionality.

In [ ]:
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

## Single-qubit quantum gates
Let us start with a single qubit and apply the Hadamard gate to it. In **LibKet** this consists of two steps:
1. Creation of a generic hadrware-agnostic **quantum expression**
2. Evaluation of the expression on a concrete **quantum backend**



### Quantum expression
Let's start with the first step and create a quantum expression that initializes the qubit to zero and apply a **Pauli-X** or **NOT**-gate to it.

In [ ]:
auto expr = measure(x(init()));

In contrast to other quantum programming languages **LibKet**'s quantum expressions have to be read from inside to outside. Note that the above expression does not perform any operations yet but only holds an abstract syntax tree (AST). You can print it with the help of the ``show(...)`` function. Let's also ignore the ``measurement(...)`` function for the time being.

In [ ]:
show<99>(expr);

Only one level of the AST is printed by default. That's why we set the print level to ``99``.

### Quantum backend
Let's evaluate our first quantum expression on a quantum simulator.

**LibKet** supports a couple of them but the easiest one is probably the open-source [QuEST](https://quest.qtechtheory.org) *Quantum Exact Simulation Toolkit* .

The following code snippet shows how to
1.  Create a quantum device for a fixed number of qubits
2.  Load our generic quantum expression into it
3.  Evaluate the quantum kernel and print the result


In [ ]:
QDevice<QDeviceType::quest, 1> device;
device(expr).eval(1);
std::cout << device.creg() << std::endl;

The result stored in the classical register ``device.creg()`` is ``1`` since $ X\lvert0\rangle=\lvert1\rangle $.

Try yourself to create an expression for $ X\lvert1\rangle $ and evaluate it.

### Measurement
Let us create another quantum expression for $ H\lvert0\rangle $, the **Hadamard** gate, and perform **measurement** afterwards.

In [ ]:
auto expr = measure(h(init()));
QDevice<QDeviceType::quest, 1> device;
device(expr).eval(1);
std::cout << device.creg() << std::endl;

The result is either $ \lvert0\rangle $ or $ \lvert1\rangle $ and changes when we run the above code snippet multiple times. Try it yourself.

If you are interested in the **probability** of ending up in either of the two states you need to remove the measurement from the quantum expression and call ``device.probabilities()``. That's of course only possible with a quantum simulator and not on a real quantum computer.

In [ ]:
auto expr = h(init());
QDevice<QDeviceType::quest, 1> device;
device(expr).eval(1);
std::cout << device.probabilities() << std::endl;

Try yourself to create an expression for $ H\lvert1\rangle $ and evaluate it.

## Single-qubit quantum circuits
Create a quantum expression for the following circuit and evaluate it

![HZH](../images/single_qubit_circuit_HZH.png)

Next create a quantum expression for the following circuit and evaluate it

![HXH](../images/single_qubit_circuit_HXH.png)

## Multi-qubit quantum circuits
To create expressions for quantum circuits with multiple qubits we need to select qubits before applying quantum gates to them. In **LibKet** this is established with the aid of **quantum filters** which act as masks or selection operators. 

![HI](../images/multi_qubit_circuit_HI.png)

A quantum expression for the above two-qubit quantum circuit can be constructed as follows

In [ ]:
auto expr = h(sel<0>(init()));

What is the result of evaluating it on the QuEST simulator? Don't forget to set the number of qubits to two.

Also try to print the expression's AST using the ``show(...)`` function to see how the filter ``sel<0>(...)`` affects it.

The ``operator<<(...)`` is overloaded for all quantum expression so that you can also print it in human-readable form

In [ ]:
std::cout << expr << std::endl;

You might have noticed the small difference with the expression you typed above? ``sel<0>`` versus ``sel_<0>``.

Filters with a trailing ``_`` are a shorthand notation for ``sel<0>(all())``, that is, application of the filter *after* resetting any previous selections. We will use this in the exercises below.

## Multi-qubit quantum gates
In the above example the multi-qubit circuit was composed of two single-qubit gates, whereby the identidy gate was left out since it had no effecty.

**LibKet** also provides a couple of true multi-qubit gates like the **CNOT**-gate. Since it operates on two qubits simultaneously we have to pass two arguments to it. The first one is the **control qubit** and the second is the **target qubit**.

In [ ]:
auto expr = cnot( sel<0>(), sel<1>( init() ) );

It is also possible to perform ``init()`` in the first argument but make sure it is not performed twice. That's why you should **never** type

In [ ]:
auto expr = cnot( sel<0>(), sel<1>() ) ( init() );

## Multi-qubit quantum circuit

Let us create a quantum expression for the following two-qubit circuit which creates the [first Bell state](https://en.wikipedia.org/wiki/Bell_state) $ \lvert \beta_{00}\rangle = (\lvert 00 \rangle + \lvert 11 \rangle) / \sqrt{2} $ 

![Bell](../images/multi_qubit_circuit_HI_CNOT.png)

After initializing both qubits to zero, a Hadamard gate is applied to the zeroth qubit followed by a **CNOT**-gate applied to both.

In [ ]:
auto expr = cnot(h(sel<0>()), sel<1>(init()));

Here we make use of the fact that filter persist after the application of gates. That is, the selection of the zeroth qubit via ``sel<0>()`` remains active after applying the Hadamard gate to it so that the control qubit is still the zeroth qubit.

## Exercise: Bell states
Write quantum expressions for the three other Bell states

$$ \lvert \beta_{01}\rangle = (\lvert 01 \rangle + \lvert 10 \rangle) / \sqrt{2} \qquad
   \lvert \beta_{10}\rangle = (\lvert 00 \rangle - \lvert 11 \rangle) / \sqrt{2} \qquad
   \lvert \beta_{11}\rangle = (\lvert 01 \rangle - \lvert 10 \rangle) / \sqrt{2} $$

You can always reset the filter selection by applying the ``all(...)`` filter. That is, ``x(sel<1>(all(h(sel<0>()))))`` applies a **Hadamard** gate on the zeroth qubit and an **X**-gate in the first qubit afterwards. Alternatively, you can use the shorthand notation ``x(sel<1>_(h(sel<0>())))``.

## Exercise: Basic quantum circuits
Write quantum expressions for the following circuits. You might need to use the ``all()`` filter or the shorthand notation ``sel_<...>`` multiple times.

![Exercise](../images/multi_qubit_circuit_exercises.png)

The second gate in the third exercise is a **SWAP**-gate, ``swap(qubit0, qubit1)`` which simply swaps the two qubits.

The last gate in the fourth exercise is termed **Toffoli** or **CCNOT**-gate, ``ccnot(ctrl0, ctrl1, target)`` which applies a **Pauli-X** or **NOT**-operation on the target qubit of both control qubits are true.

## Application: Quantum teleportation
At the end of this tutorial we have a look at a famous quantum algorithm - [Quantum Teleportation](https://en.wikipedia.org/wiki/Quantum_teleportation). In short, Alice wants to send the qubit state $ \lvert q_0\rangle = \alpha \lvert0\rangle+\beta \lvert1\rangle$ to Bob. The no-cloning theorem forbids her to make a simple copy of the unknown quantum state. What she can do is to share with Bob an entangled qubit pair and transfer the information via two classical bits. This, of course, involves two measurements.

![Quantum teleportation](../images/quantum_teleportation.png)

Implement a **LibKet** expression for the above quantum circuit. Start with $ \lvert q_1\rangle = \lvert q_2\rangle:=\lvert0\rangle $ and test different values for Alice's message $\lvert q_0\rangle $.